# Complément : Charger et compresser une vraie image

**Niveau :** Première NSI

Ce notebook complète le TP principal en montrant comment appliquer la transformation de Haar sur de **vraies images**.

## 📸 Trois façons de charger une image :

1. **Image de démonstration** (déjà intégrée au notebook)
2. **Upload d'une image** (bouton d'upload dans Capytale)
3. **Charger depuis un fichier** (si vous avez une image dans le même dossier)

## Installation et importations

In [ ]:
# Bibliothèques nécessaires
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import io
import base64

# Pour l'upload de fichiers (Capytale)
try:
    from ipywidgets import FileUpload, Button, Output, VBox
    from IPython.display import display
    UPLOAD_DISPONIBLE = True
except ImportError:
    UPLOAD_DISPONIBLE = False
    print("⚠️ ipywidgets non disponible. L'upload ne fonctionnera pas.")
    print("   Utilisez l'image de démonstration ou chargez depuis un fichier.")

print("✓ Bibliothèques chargées")

## Fonctions de transformation de Haar

On réutilise les fonctions du TP principal.

In [ ]:
def transformation_haar_2x2(carre):
    """Transformation de Haar sur un carré 2x2."""
    a, b = carre[0, 0], carre[0, 1]
    c, d = carre[1, 0], carre[1, 1]
    
    rouge = (a + b + c + d) / 4
    bleu = ((b - a) + (d - c)) / 4
    vert = ((c - a) + (d - b)) / 4
    violet = ((b + c) - (a + d)) / 4
    
    return np.array([[rouge, bleu], [vert, violet]])

def transformation_haar_image(image):
    """Transformation de Haar sur une image complète."""
    hauteur, largeur = image.shape
    nouvelle_hauteur = hauteur // 2
    nouvelle_largeur = largeur // 2
    
    rouge = np.zeros((nouvelle_hauteur, nouvelle_largeur))
    bleu = np.zeros((nouvelle_hauteur, nouvelle_largeur))
    vert = np.zeros((nouvelle_hauteur, nouvelle_largeur))
    violet = np.zeros((nouvelle_hauteur, nouvelle_largeur))
    
    for i in range(0, hauteur, 2):
        for j in range(0, largeur, 2):
            bloc = image[i:i+2, j:j+2]
            transforme = transformation_haar_2x2(bloc)
            
            rouge[i//2, j//2] = transforme[0, 0]
            bleu[i//2, j//2] = transforme[0, 1]
            vert[i//2, j//2] = transforme[1, 0]
            violet[i//2, j//2] = transforme[1, 1]
    
    resultat = np.zeros_like(image)
    resultat[0:nouvelle_hauteur, 0:nouvelle_largeur] = rouge
    resultat[0:nouvelle_hauteur, nouvelle_largeur:largeur] = bleu
    resultat[nouvelle_hauteur:hauteur, 0:nouvelle_largeur] = vert
    resultat[nouvelle_hauteur:hauteur, nouvelle_largeur:largeur] = violet
    
    return resultat, rouge, bleu, vert, violet

def transformation_haar_multi_niveaux(image, niveaux=3):
    """Transformation de Haar à plusieurs niveaux."""
    resultat = image.copy()
    hauteur, largeur = image.shape
    
    for niveau in range(niveaux):
        h = hauteur // (2 ** niveau)
        l = largeur // (2 ** niveau)
        
        if h < 2 or l < 2:
            break
            
        quadrant = resultat[0:h, 0:l]
        transforme, _, _, _, _ = transformation_haar_image(quadrant)
        resultat[0:h, 0:l] = transforme
    
    return resultat

def compresser(image_transformee, seuil=10):
    """Compression par seuillage."""
    compresse = image_transformee.copy()
    masque = np.abs(compresse) < seuil
    compresse[masque] = 0
    return compresse

print("✓ Fonctions de transformation chargées")

## Fonctions utilitaires pour les images

In [ ]:
def pil_vers_array_gris(image_pil):
    """
    Convertit une image PIL en tableau numpy niveaux de gris.
    """
    # Convertir en niveaux de gris
    image_gris = image_pil.convert('L')
    
    # Convertir en array numpy
    array = np.array(image_gris, dtype=float)
    
    return array

def redimensionner_puissance_2(image_pil, taille=256):
    """
    Redimensionne une image PIL à une taille carrée (puissance de 2).
    Utilise un crop centré puis un resize.
    """
    # Recadrer en carré (crop centré)
    largeur, hauteur = image_pil.size
    cote = min(largeur, hauteur)
    
    left = (largeur - cote) // 2
    top = (hauteur - cote) // 2
    right = left + cote
    bottom = top + cote
    
    image_carree = image_pil.crop((left, top, right, bottom))
    
    # Redimensionner
    image_redim = image_carree.resize((taille, taille), Image.Resampling.LANCZOS)
    
    return image_redim

def analyser_et_afficher(image_pil, titre="Image", niveaux=5):
    """
    Analyse complète d'une image : conversion, transformation, affichage.
    """
    print(f"\n{'='*60}")
    print(f"Analyse de : {titre}")
    print(f"{'='*60}")
    
    # Redimensionner
    img_redim = redimensionner_puissance_2(image_pil, taille=256)
    print(f"✓ Image redimensionnée : {img_redim.size}")
    
    # Convertir en niveaux de gris
    img_array = pil_vers_array_gris(img_redim)
    print(f"✓ Conversion en niveaux de gris : {img_array.shape}")
    
    # Transformer
    img_trans = transformation_haar_multi_niveaux(img_array, niveaux=niveaux)
    print(f"✓ Transformation de Haar : {niveaux} niveaux")
    
    # Affichage
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    
    # Image originale (couleur)
    axes[0, 0].imshow(image_pil)
    axes[0, 0].set_title(f"{titre}\n(Originale couleur)", fontsize=14, fontweight='bold')
    axes[0, 0].axis('off')
    
    # Image en niveaux de gris
    axes[0, 1].imshow(img_array, cmap='gray', vmin=0, vmax=255)
    axes[0, 1].set_title("Niveaux de gris\n256×256", fontsize=14)
    axes[0, 1].axis('off')
    
    # Image transformée
    axes[0, 2].imshow(img_trans, cmap='gray')
    axes[0, 2].set_title(f"Transformée de Haar\n({niveaux} niveaux)", fontsize=14)
    axes[0, 2].axis('off')
    
    # Compressions avec différents seuils
    seuils = [10, 30, 50]
    for i, seuil in enumerate(seuils):
        img_comp = compresser(img_trans, seuil)
        nb_zeros = np.sum(img_comp == 0)
        pct = (nb_zeros / img_comp.size) * 100
        
        axes[1, i].imshow(img_comp, cmap='gray')
        axes[1, i].set_title(f"Compression seuil={seuil}\n{pct:.1f}% de zéros", fontsize=12)
        axes[1, i].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # Statistiques
    coeffs = img_trans.flatten()
    print(f"\n📊 Statistiques :")
    print(f"   Coefficients totaux : {len(coeffs)}")
    print(f"   Moyenne : {np.mean(coeffs):.2f}")
    print(f"   Écart-type : {np.std(coeffs):.2f}")
    print(f"   |Coeff| < 10 : {(np.abs(coeffs) < 10).sum() / len(coeffs) * 100:.1f}%")
    print(f"   |Coeff| < 30 : {(np.abs(coeffs) < 30).sum() / len(coeffs) * 100:.1f}%")
    print(f"   |Coeff| < 50 : {(np.abs(coeffs) < 50).sum() / len(coeffs) * 100:.1f}%")

print("✓ Fonctions utilitaires chargées")

---

# Méthode 1 : Image de démonstration intégrée 🏠

Cette image est **déjà dans le notebook**, pas besoin de fichier externe !

In [ ]:
def creer_image_maison():
    """
    Crée une image de démonstration : une maison simple.
    """
    # Créer une image 256x256
    img = Image.new('RGB', (256, 256), color=(135, 206, 235))  # Fond bleu ciel
    pixels = img.load()
    
    # Herbe verte
    for y in range(180, 256):
        for x in range(256):
            pixels[x, y] = (34, 139, 34)
    
    # Mur de la maison (beige)
    for y in range(120, 180):
        for x in range(80, 176):
            pixels[x, y] = (245, 222, 179)
    
    # Toit (rouge)
    for y in range(60, 120):
        debut = 80 - (120 - y)
        fin = 176 + (120 - y)
        for x in range(max(0, debut), min(256, fin)):
            pixels[x, y] = (178, 34, 34)
    
    # Porte (marron)
    for y in range(150, 180):
        for x in range(110, 140):
            pixels[x, y] = (101, 67, 33)
    
    # Fenêtre gauche (bleu clair)
    for y in range(130, 150):
        for x in range(90, 110):
            pixels[x, y] = (173, 216, 230)
    
    # Fenêtre droite (bleu clair)
    for y in range(130, 150):
        for x in range(146, 166):
            pixels[x, y] = (173, 216, 230)
    
    # Soleil (jaune)
    centre_x, centre_y = 220, 40
    rayon = 15
    for y in range(centre_y - rayon, centre_y + rayon):
        for x in range(centre_x - rayon, centre_x + rayon):
            if (x - centre_x)**2 + (y - centre_y)**2 <= rayon**2:
                if 0 <= x < 256 and 0 <= y < 256:
                    pixels[x, y] = (255, 255, 0)
    
    return img

# Créer l'image de démonstration
image_demo = creer_image_maison()

print("✓ Image de démonstration créée")
print(f"  Taille : {image_demo.size}")
print(f"  Mode : {image_demo.mode}")

# Afficher l'image
plt.figure(figsize=(6, 6))
plt.imshow(image_demo)
plt.title("Image de démonstration : Maison", fontsize=16, fontweight='bold')
plt.axis('off')
plt.show()

### Analyser l'image de démonstration

In [ ]:
# Analyser l'image de démonstration
analyser_et_afficher(image_demo, titre="Maison (démonstration)", niveaux=6)

---

# Méthode 2 : Upload d'une image 📤

**Pour Capytale :** Utilisez le bouton ci-dessous pour téléverser votre propre image !

In [ ]:
if UPLOAD_DISPONIBLE:
    # Créer le widget d'upload
    upload_widget = FileUpload(
        accept='image/*',  # N'accepte que les images
        multiple=False,  # Une seule image à la fois
        description='Choisir une image'
    )
    
    # Zone de sortie pour l'affichage
    output = Output()
    
    def traiter_upload(change):
        """Traite l'image uploadée."""
        with output:
            output.clear_output()
            
            if len(upload_widget.value) == 0:
                print("❌ Aucune image sélectionnée")
                return
            
            # Récupérer l'image
            uploaded_file = list(upload_widget.value.values())[0]
            nom_fichier = list(upload_widget.value.keys())[0]
            
            print(f"📁 Fichier reçu : {nom_fichier}")
            print(f"   Taille : {len(uploaded_file['content'])} octets")
            
            try:
                # Charger l'image avec PIL
                image_bytes = io.BytesIO(uploaded_file['content'])
                image_uploadee = Image.open(image_bytes)
                
                print(f"✓ Image chargée : {image_uploadee.size} pixels, mode {image_uploadee.mode}")
                
                # Analyser
                analyser_et_afficher(image_uploadee, titre=nom_fichier, niveaux=5)
                
            except Exception as e:
                print(f"❌ Erreur lors du chargement : {e}")
    
    # Lier le traitement à l'upload
    upload_widget.observe(traiter_upload, names='value')
    
    # Afficher le widget
    display(VBox([upload_widget, output]))
    
    print("\n👆 Cliquez sur 'Choisir une image' ci-dessus pour uploader votre photo !")
    print("   Formats supportés : JPG, PNG, GIF, BMP, etc.")
    
else:
    print("❌ Le widget d'upload n'est pas disponible.")
    print("   Utilisez la Méthode 1 (image de démo) ou la Méthode 3 (fichier local).")

---

# Méthode 3 : Charger depuis un fichier local 📂

Si vous avez une image dans le même dossier que le notebook, utilisez cette méthode.

In [ ]:
# TODO: Modifiez le nom du fichier ici
FICHIER_IMAGE = 'mon_image.jpg'  # ← Remplacez par votre nom de fichier

try:
    # Charger l'image avec PIL
    image_fichier = Image.open(FICHIER_IMAGE)
    
    print(f"✓ Image chargée : {FICHIER_IMAGE}")
    print(f"  Taille : {image_fichier.size}")
    print(f"  Mode : {image_fichier.mode}")
    
    # Analyser
    analyser_et_afficher(image_fichier, titre=FICHIER_IMAGE, niveaux=5)
    
except FileNotFoundError:
    print(f"❌ Fichier '{FICHIER_IMAGE}' introuvable !")
    print("\n💡 Solutions :")
    print("   1. Vérifiez le nom du fichier (respectez les majuscules/minuscules)")
    print("   2. Vérifiez que l'image est dans le même dossier que ce notebook")
    print("   3. Utilisez la Méthode 1 (image de démo) ou la Méthode 2 (upload)")
    
except Exception as e:
    print(f"❌ Erreur : {e}")

---

# 🔬 Analyse détaillée des composantes

Regardons en détail les 4 quadrants après la transformation.

In [ ]:
# Utiliser l'image de démonstration pour cette analyse
img_array = pil_vers_array_gris(redimensionner_puissance_2(image_demo, 256))

# Première transformation seulement
_, rouge, bleu, vert, violet = transformation_haar_image(img_array)

fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# Image originale
axes[0, 0].imshow(img_array, cmap='gray', vmin=0, vmax=255)
axes[0, 0].set_title("Image originale\n256×256 pixels", fontsize=16, fontweight='bold')
axes[0, 0].axis('off')

# Rouge (structure)
axes[0, 1].imshow(rouge, cmap='gray', vmin=0, vmax=255)
axes[0, 1].set_title("🔴 ROUGE : Moyennes\n(Structure générale 128×128)", fontsize=14)
axes[0, 1].axis('off')

# Statistiques
axes[0, 2].axis('off')
stats_text = f"""STATISTIQUES DES QUADRANTS

🔴 ROUGE (moyennes):
  Min: {rouge.min():.1f}
  Max: {rouge.max():.1f}
  Moyenne: {rouge.mean():.1f}

🔵 BLEU (vertical):
  Min: {bleu.min():.1f}
  Max: {bleu.max():.1f}
  Moy abs: {np.abs(bleu).mean():.1f}

🟢 VERT (horizontal):
  Min: {vert.min():.1f}
  Max: {vert.max():.1f}
  Moy abs: {np.abs(vert).mean():.1f}

🟣 VIOLET (oblique):
  Min: {violet.min():.1f}
  Max: {violet.max():.1f}
  Moy abs: {np.abs(violet).mean():.1f}
"""
axes[0, 2].text(0.1, 0.5, stats_text, fontsize=12, family='monospace',
                verticalalignment='center')

# Bleu (contours verticaux)
axes[1, 0].imshow(np.abs(bleu), cmap='hot', vmin=0, vmax=50)
axes[1, 0].set_title("🔵 BLEU : Différences verticales\n(Contours verticaux)", fontsize=14)
axes[1, 0].axis('off')

# Vert (contours horizontaux)
axes[1, 1].imshow(np.abs(vert), cmap='hot', vmin=0, vmax=50)
axes[1, 1].set_title("🟢 VERT : Différences horizontales\n(Contours horizontaux)", fontsize=14)
axes[1, 1].axis('off')

# Violet (contours obliques)
axes[1, 2].imshow(np.abs(violet), cmap='hot', vmin=0, vmax=50)
axes[1, 2].set_title("🟣 VIOLET : Différences obliques\n(Contours diagonaux)", fontsize=14)
axes[1, 2].axis('off')

plt.suptitle("Décomposition en ondelettes de Haar", fontsize=18, fontweight='bold', y=0.98)
plt.tight_layout()
plt.show()

print("\n💡 Observations :")
print("   - Les zones SOMBRES dans bleu/vert/violet = peu de changement (compressible !)")
print("   - Les zones CLAIRES = changements importants (contours, détails à garder)")
print("   - Le quadrant ROUGE contient l'essentiel de l'information visuelle")

---

# 📊 Comparaison de différentes images

Créons plusieurs types d'images pour comparer leur compressibilité.

In [ ]:
def creer_image_uniforme():
    """Crée une image très uniforme (facile à compresser)."""
    img = Image.new('RGB', (256, 256), color=(100, 150, 200))
    return img

def creer_image_damier():
    """Crée un damier (difficile à compresser)."""
    img = Image.new('L', (256, 256))
    pixels = img.load()
    taille_case = 16
    for y in range(256):
        for x in range(256):
            if ((x // taille_case) + (y // taille_case)) % 2 == 0:
                pixels[x, y] = 255
            else:
                pixels[x, y] = 0
    return img.convert('RGB')

def creer_image_degrade():
    """Crée un dégradé (compression moyenne)."""
    img = Image.new('L', (256, 256))
    pixels = img.load()
    for y in range(256):
        for x in range(256):
            pixels[x, y] = x
    return img.convert('RGB')

# Créer les images
images_test = {
    'Uniforme (ciel)': creer_image_uniforme(),
    'Dégradé': creer_image_degrade(),
    'Damier': creer_image_damier(),
    'Maison': image_demo
}

# Afficher toutes les images
fig, axes = plt.subplots(1, 4, figsize=(20, 5))

for ax, (nom, img) in zip(axes, images_test.items()):
    ax.imshow(img)
    ax.set_title(nom, fontsize=14, fontweight='bold')
    ax.axis('off')

plt.suptitle("Images de test", fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# Comparer les taux de compression
resultats = {}

for nom, img in images_test.items():
    img_redim = redimensionner_puissance_2(img, 256)
    img_array = pil_vers_array_gris(img_redim)
    img_trans = transformation_haar_multi_niveaux(img_array, niveaux=5)
    
    # Calculer le taux de compression pour différents seuils
    taux = []
    for seuil in [10, 20, 30, 40, 50]:
        img_comp = compresser(img_trans, seuil)
        pct = (np.sum(img_comp == 0) / img_comp.size) * 100
        taux.append(pct)
    
    resultats[nom] = taux

# Tracer les courbes
plt.figure(figsize=(12, 7))
seuils = [10, 20, 30, 40, 50]

for nom, taux in resultats.items():
    plt.plot(seuils, taux, marker='o', linewidth=2, markersize=8, label=nom)

plt.xlabel("Seuil de compression", fontsize=14)
plt.ylabel("Pourcentage de coefficients à zéro (%)", fontsize=14)
plt.title("Compressibilité de différentes images", fontsize=16, fontweight='bold')
plt.legend(fontsize=12, loc='lower right')
plt.grid(True, alpha=0.3)
plt.xlim(5, 55)
plt.ylim(0, 100)
plt.show()

print("\n📈 Analyse :")
print("   - L'image UNIFORME se compresse très bien (beaucoup de zéros)")
print("   - Le DAMIER se compresse mal (beaucoup de détails haute fréquence)")
print("   - Le DÉGRADÉ et la MAISON sont intermédiaires")

---

# 🎯 À vous de jouer !

## Exercices suggérés :

1. **Uploadez votre photo** et analysez-la
2. **Comparez** : une photo de paysage vs une photo de portrait
3. **Trouvez le seuil optimal** : meilleur compromis qualité/compression
4. **Créez votre propre image** avec PIL et testez sa compression

## Questions de réflexion :

- Pourquoi le ciel bleu se compresse-t-il mieux qu'un damier ?
- Quel type de photo se compresse le mieux : portrait ou paysage ?
- À partir de quel seuil perd-on trop de qualité ?

---

**TP JPEG 2000 - 1ère NSI**  
**Notebook complémentaire : Charger des images avec PIL**